In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np


In [72]:
df= pd.read_pickle("train_test_data_binary.pkl.bz2",compression="bz2")

In [73]:
df.head().T

,0,1,2,3,4
member_id,1296599.0,1293387.0,1239172.0,1215683.0,1228992.0
annual_inc,24000.0,49000.0,47000.0,80000.0,106000.0
age,56.0,55.0,31.0,54.0,28.0
default,0.0,0.0,0.0,0.0,0.0
loan,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
mf_sub_pref_ultra_short_duration,0.0,0.0,0.0,0.0,0.0
mf_sub_pref_value,0.0,0.0,0.0,0.0,0.0
loan_status_charged_off,0.0,0.0,0.0,0.0,0.0
loan_status_current,0.0,0.0,0.0,0.0,1.0


In [75]:
cols_to_drop=["member_id"]

In [76]:
df.columns

Index(['member_id', 'annual_inc', 'age', 'default', 'loan', 'housing', 'mf_id',
       'amount', 'tenure_yr', 'home_ownership_mortgage', 'home_ownership_none',
       'home_ownership_other', 'home_ownership_own', 'home_ownership_rent',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'risk_high', 'risk_low', 'risk_low_to_moderate',
       'risk_moderate', 'risk_moderately_high', 'risk_very_high',
       'job_accounting', 'job_architects', 'job_aviation',
       'job_chartered_accountant', 'job_defence', 'job_entrepreneurship',
       'job_finance', 'job_government', 'job_healthcare',
       'job_human_resources', 'job_journalism', 'job_legal', 'job_marketing',
       'job_ngo', 'job_operations', 'job_others', 'job_teacher',
    

In [77]:
df.drop(columns=cols_to_drop,axis=1,inplace=True)

In [78]:
df = df.reset_index(drop=True)

In [79]:
df.head().T

,0,1,2,3,4
annual_inc,24000.0,49000.0,47000.0,80000.0,106000.0
age,56.0,55.0,31.0,54.0,28.0
default,0.0,0.0,0.0,0.0,0.0
loan,0.0,0.0,0.0,0.0,0.0
housing,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
mf_sub_pref_ultra_short_duration,0.0,0.0,0.0,0.0,0.0
mf_sub_pref_value,0.0,0.0,0.0,0.0,0.0
loan_status_charged_off,0.0,0.0,0.0,0.0,0.0
loan_status_current,0.0,0.0,0.0,0.0,1.0


In [80]:
scheme_names=df[""].nunique()
scheme_names

13882

In [81]:
scheme_list = list(set(df["mf_id"]))

In [82]:
df["scheme_code"]=df["mf_id"].astype("category").cat.codes

In [83]:
mf_cat_codes=pd.DataFrame(df[["scheme_code","mf_id"]].value_counts().reset_index(),columns=["scheme_code","mf_id","count"])

In [84]:
mf_cat_codes

,scheme_code,mf_id,count
0,12694,147675,NaN
1,11729,146155,NaN
2,8892,141410,NaN
3,4057,123689,NaN
4,8985,141573,NaN
...,...,...,...
13877,12292,147134,NaN
13878,9655,142727,NaN
13879,2541,119127,NaN
13880,7852,139341,NaN


In [85]:
df.drop("mf_id",axis=1,inplace=True)

In [86]:
mf_cat_codes=mf_cat_codes[["scheme_code","mf_id"]].drop_duplicates()

In [87]:
mf_cat_codes.shape

(13882, 2)

In [89]:
X=df.drop(columns="scheme_code",axis=1)
y=df["scheme_code"]

In [90]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

In [91]:
# Test ,Train split
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7,test_size=0.3,random_state=100)

In [94]:
X.shape[1]

93

In [95]:
X_train=scaler.fit_transform(X_train)

In [96]:
input_shape=X_train.shape

output_shape=y_train.shape

In [97]:
input_shape

(263935, 93)

In [98]:
output_shape

(263935,)

In [99]:
def one_hot(j):
    # input is the target dataset of shape (1, m) where m is the number of data points
    # returns a 2 dimensional array of shape (10, m) where each target value is converted to a one hot encoding
    # Look at the next block of code for a better understanding of one hot encoding
    n = j.shape[0]
    new_array = np.zeros((scheme_names, n))
    index = 0
    for res in j:
        new_array[res][index] = 1.0
        index = index + 1
    return new_array

In [100]:
y_train.shape

(263935,)

In [101]:
y_train_enc=one_hot(y_train)
y_train_enc

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [102]:
y_train_enc.shape

(13882, 263935)

In [103]:
y_train =y_train_enc.T

In [104]:
X_train.shape

(263935, 93)

In [105]:
output=y_train.shape[1]
output

13882

In [35]:
# !pip install -q keras

In [106]:
from tensorflow import keras 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation ,Flatten
from tensorflow.keras import regularizers

In [139]:
# create model
nn_model = Sequential()
nn_model.add(Dense(1024, input_dim=X.shape[1], activation='relu'))
nn_model.add(Dropout(rate=0.3))
nn_model.add(BatchNormalization())
nn_model.add(Dense(512, activation = 'relu'))
nn_model.add(Dropout(rate=0.3))
nn_model.add(BatchNormalization())
nn_model.add(Dense(256, activation = 'relu'))
nn_model.add(Dropout(rate=0.3))
nn_model.add(BatchNormalization())
nn_model.add(Dense(128, activation = 'relu'))
nn_model.add(Dropout(rate=0.3))
nn_model.add(BatchNormalization())
nn_model.add(Dense(64, activation = 'relu'))
nn_model.add(Dropout(rate=0.3))
nn_model.add(BatchNormalization())
nn_model.add(Dense(32, activation = 'relu'))
nn_model.add(Dense(output, activation='softmax'))

In [140]:
nn_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 1024)              96256     
_________________________________________________________________
dropout_20 (Dropout)         (None, 1024)              0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 1024)              4096      
_________________________________________________________________
dense_29 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_21 (Dropout)         (None, 512)               0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 512)               2048      
_________________________________________________________________
dense_30 (Dense)             (None, 256)              

In [141]:
#!pip install tensorflow-gpu --upgrade

In [118]:
METRICS = [  
      keras.metrics.AUC(name='auc')
]

In [143]:
nn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)

In [ ]:
nn_model.fit(X_train, y_train, epochs=30, batch_size=100)

Epoch 1/30
2640/2640 [==============================] - 786s 298ms/step - loss: 8.5724 - auc: 0.5001
Epoch 2/30
2640/2640 [==============================] - 786s 298ms/step - loss: 7.4327 - auc: 0.5062
Epoch 3/30
2640/2640 [==============================] - 787s 298ms/step - loss: 7.0258 - auc: 0.5301
Epoch 4/30
2640/2640 [==============================] - 787s 298ms/step - loss: 6.7552 - auc: 0.5592
Epoch 5/30
2640/2640 [==============================] - 786s 298ms/step - loss: 6.5839 - auc: 0.5825
Epoch 6/30
2640/2640 [==============================] - 787s 298ms/step - loss: 6.4857 - auc: 0.6015
Epoch 7/30
2640/2640 [==============================] - 786s 298ms/step - loss: 6.4229 - auc: 0.6144
Epoch 8/30
2640/2640 [==============================] - 786s 298ms/step - loss: 6.3765 - auc: 0.6236
Epoch 9/30
2640/2640 [==============================] - 786s 298ms/step - loss: 6.3289 - auc: 0.6317
Epoch 10/30
2640/2640 [==============================] - 787s 298ms/step - loss: 6.2863 - a

In [ ]:
!pip install pyyaml h5py

In [147]:
test=X_test[:1]

In [160]:
predictions = nn_model.predict(test)

In [161]:
predictions

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [162]:
predictions.shape

(1, 13882)

In [163]:
nn_model.save("content_based_recommendar_model.h5")

In [164]:
from tensorflow.keras.models import load_model
# load model
model = load_model("content_based_recommendar_model.h5")
# summarize model.
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 1024)              96256     
_________________________________________________________________
dropout_20 (Dropout)         (None, 1024)              0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 1024)              4096      
_________________________________________________________________
dense_29 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_21 (Dropout)         (None, 512)               0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 512)               2048      
_________________________________________________________________
dense_30 (Dense)             (None, 256)              

In [165]:
predictions = model.predict(test)

In [166]:
np.max(predictions)

1.0

In [167]:
predictions.shape

(1, 13882)

In [153]:
predictions_index = int(np.argmax(predictions, axis = 1))

In [154]:
predictions_index

5575

In [155]:
predictions.T[predictions_index]

array([1.], dtype=float32)

In [156]:
predictions_index

5575

In [157]:
scheme_list[predictions_index]

143828

In [67]:
scheme_list[0]

131076

In [68]:
mf_result.head()

NameError: name 'mf_result' is not defined

In [ ]:
mf_result.drop_duplicates(keep="first",inplace=True)
mf_result = mf_result.reset_index(drop=True)
mf_result.to_csv("mf_index.csv.zip",compression="zip",index=False)

In [ ]:
# mf_result = pd.read_csv("mf_index.csv.zip")

In [ ]:
mf_result[mf_result["index"]==predictions_index]

In [168]:
top_5=(-predictions.ravel()).argsort()[:5]

In [169]:
top_5

array([1721,    0, 9247, 9248, 9249])

In [138]:
mf_cat_codes.to_pickle("mf_output_mapping.pkl.bz2",compression="bz2")

In [ ]:
mf_cat_codes[mf_cat_codes["scheme_code"].isin(top_5)]